In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from dask.diagnostics import ProgressBar
from geocube.api.core import make_geocube

PATH = "D:\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"

c:\Users\ofici\AppData\Local\Programs\Python\Python311\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [3]:
gdf = gpd.read_file(r"D:\Datasets\International Best Track Archive for Climate Stewardship (IBTrACS)\IBTrACS.ALL.list.v04r01.lines.shp")

In [4]:
gdf

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEA_SW,USA_SEA_NW,STORM_SPD,STORM_DR,year,month,day,hour,min,geometry
0,1842298N11080,1842,1,NI,BB,UNNAMED,1842-10-25 03:00:00,NR,10.9,80.3,...,NaN,NaN,9,265,1842,10,25,3,0,"LINESTRING (80.30005 10.90000, 79.80005 10.90000)"
1,1842298N11080,1842,1,NI,BB,UNNAMED,1842-10-25 06:00:00,NR,10.9,79.8,...,NaN,NaN,9,265,1842,10,25,6,0,"LINESTRING (79.80005 10.90000, 79.40002 10.80000)"
2,1842298N11080,1842,1,NI,BB,UNNAMED,1842-10-25 09:00:00,NR,10.8,79.4,...,NaN,NaN,9,265,1842,10,25,9,0,"LINESTRING (79.40002 10.80000, 78.90002 10.80000)"
3,1842298N11080,1842,1,NI,BB,UNNAMED,1842-10-25 12:00:00,NR,10.8,78.9,...,NaN,NaN,9,265,1842,10,25,12,0,"LINESTRING (78.90002 10.80000, 78.40002 10.80000)"
4,1842298N11080,1842,1,NI,BB,UNNAMED,1842-10-25 15:00:00,NR,10.8,78.4,...,NaN,NaN,9,270,1842,10,25,15,0,"LINESTRING (78.40002 10.80000, 77.90002 10.80000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698802,2024178N09335,2024,30,NA,NA,BERYL,2024-07-01 09:00:00,TS,11.7,-59.7,...,90.0,180.0,16,290,2024,7,1,9,0,"LINESTRING (-59.69995 11.70000, -60.50000 12.0..."
698803,2024178N09335,2024,30,NA,CS,BERYL,2024-07-01 12:00:00,TS,12.0,-60.5,...,105.0,180.0,17,295,2024,7,1,12,0,"LINESTRING (-60.50000 12.00000, -61.40002 12.4..."
698804,2024178N09335,2024,30,NA,CS,BERYL,2024-07-01 15:00:00,TS,12.4,-61.4,...,90.0,195.0,19,295,2024,7,1,15,0,"LINESTRING (-61.40002 12.40000, -62.30005 12.8..."
698805,2024178N09335,2024,30,NA,CS,BERYL,2024-07-01 18:00:00,TS,12.8,-62.3,...,75.0,210.0,19,295,2024,7,1,18,0,"LINESTRING (-62.30005 12.80000, -63.19995 13.1..."


In [3]:
ds = xr.open_dataset(r"D:\Datasets\The International Best Track Archive for Climate Stewardship (IBTrACS)\IBTrACS.ALL.v04r01.nc")

In [20]:
ds.isel(storm=1500)

<xarray.Dataset> Size: 280kB
Dimensions:           (date_time: 360, quadrant: 4)
Coordinates:
    time              (date_time) datetime64[ns] 3kB ...
    lat               (date_time) float32 1kB ...
    lon               (date_time) float32 1kB ...
Dimensions without coordinates: date_time, quadrant
Data variables: (12/159)
    numobs            float32 4B ...
    sid               |S13 13B ...
    season            float32 4B ...
    number            int16 2B ...
    basin             (date_time) |S2 720B ...
    subbasin          (date_time) |S2 720B ...
    ...                ...
    reunion_gust      (date_time) float32 1kB ...
    reunion_gust_per  (date_time) float32 1kB ...
    usa_seahgt        (date_time) float32 1kB ...
    usa_searad        (date_time, quadrant) float32 6kB ...
    storm_speed       (date_time) float32 1kB ...
    storm_dir         (date_time) float32 1kB ...
Attributes: (12/49)
    title:                      IBTrACS - International Best Track Archive fo...
    summary:                    The intent of the IBTrACS project is to overc...
    source:                     The original data are tropical cyclone positi...
    Conventions:                ACDD-1.3
    Conventions_note:           Data are nearly CF-1.7 compliant. The sole is...
    product_version:            v04r01
    ...                         ...
    history:                    Tue Jul  2 06:44:42 2024: ncks --no_abc --cnk...
    license:                    These data may be redistributed and used with...
    featureType:                trajectory
    cdm_data_type:              Trajectory
    comment:                    The tracks of TCs generally look like a traje...
    NCO:                        netCDF Operators version 4.8.1 (Homepage = ht...